# **rinna版GPT-2のテキスト生成(Ubuntu)** 

# GPUが使えるか確認

In [ ]:
import torch
print(torch.cuda.is_available())

In [ ]:
!git clone https://github.com/huggingface/transformers 

In [ ]:
!pip install transformers==4.5.0
!pip install datasets==1.2.1
!pip install sentencepiece==0.1.91
!pip install ipadic

# **run_clm.pyの変更**
```
 ./transformers/examples/language-modeling/run_clm.py
 ```
## 追加
```
from transformers import T5Tokenizer
```
## 変更
```
check_min_version("4.5.0")
tokenizer = T5Tokenizer.from_pretrained(model_args.tokenizer_name, **tokenizer_kwargs)
tokenizer = T5Tokenizer.from_pretrained(model_args.model_name_or_path, **tokenizer_kwargs)
```


## **ファインチューニングの実行**
- train.txt

## 学習用

In [ ]:
%%time

# 事前学習の実行
!python ./transformers/examples/pytorch/language-modeling/run_clm.py \
    --model_name_or_path=rinna/japanese-gpt2-medium \
    --train_file=train_new.txt \
    --validation_file=train.txt \
    --do_train \
    --do_eval \
    --num_train_epochs=3 \
    --save_steps=10000 \
    --save_total_limit=3 \
    --per_device_train_batch_size=1 \
    --per_device_eval_batch_size=1 \
    --output_dir=chat/ \
    --use_fast_tokenizer=False

## AutoModelForCausal

In [ ]:
import torch
import torch.cuda.amp as amp
from transformers import T5Tokenizer, AutoModelForCausalLM, TFAutoModelForCausalLM, AutoTokenizer
import csv


#入力文字
enter = ['<s>暑すぎる[SEP]']
name = 'test'
num = 1

header = ["入力","出力"]
"""
with open("./csv/{0}.csv".format(name), 'w', encoding="sjis") as f:
    writer = csv.writer(f, lineterminator="\n")
    writer.writerow(header)
"""

# トークナイザーとモデルの準備
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-medium")
#スペシャルトークン
#print(tokenizer.special_tokens_map)
#print(tokenizer.all_special_ids)

#tokenizer = AutoTokenizer.from_pretrained("rinna/japanese-gpt2-medium")
#tokenizer = transformers.BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")

#モデル
#model = AutoModelForCausalLM.from_pretrained("chat_new_5/")
model = AutoModelForCausalLM.from_pretrained("kaedefuto/chat_bot")
#pt_model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt2-medium")

"""

for sentence in enter:
    #print(num)
    # 推論
    pt_tensor = tokenizer.encode(sentence, return_tensors="pt")
    #pt_tensor = tokenizer.encode(sentence, return_tensors="pt").to(device)
    #print(pt_tensor)
    output = model.generate(
        input_ids=pt_tensor,
        do_sample=True,
        top_p=1.0,
        top_k=50,
        early_stopping=True,
        max_length=50,
        min_length=10,
        num_return_sequences=num,
        output_scores=True,
        bad_words_ids=[[1], [5]]
    )
    outer = tokenizer.batch_decode(output)

    #</s>の削除
    outer = [s.replace('</s> ','') for s in outer]
    outer = [s.replace('</s>','') for s in outer]
    outer = [s.replace('<unk> ','') for s in outer]
    outer = [s.replace('<unk>','') for s in outer] 
    length = len(outer)
    print(outer) 


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
"""

with amp.autocast():
    for sentence in enter:
        #print(num)
        # 推論
        pt_tensor = tokenizer.encode(sentence, return_tensors="pt")
        #pt_tensor = tokenizer.encode(sentence, return_tensors="pt").to(device)
        #print(pt_tensor)
        output = model.generate(
            input_ids=pt_tensor,
            do_sample=True,
            top_p=1.0,
            top_k=50,
            early_stopping=True,
            max_length=50,
            min_length=10,
            num_return_sequences=num,
            output_scores=True,
            bad_words_ids=[[1], [5]]
        )
        outer = tokenizer.batch_decode(output)

        #</s>の削除
        outer = [s.replace('</s> ','') for s in outer]
        outer = [s.replace('<unk>','') for s in outer] 
        length = len(outer)
        #print(outer) 
        
        
        for i in outer:
            print(i)
            #print("\n")
 
        
        """
        for i in range(length):
            outer[i] = outer[i].replace('</s>','')
            print(outer)
        """
        

        #CSV書き込み
        """
        with open("./csv/{0}.csv".format(name), 'a', encoding="sjis") as f:
            writer = csv.writer(f, lineterminator="\n")
            one_row = [sentence, '']
            writer.writerow(one_row)
            for i in range(length):
                one_row = ["", outer[i]]
                  writer.writerow(one_row)
        num += 1
        """

print('complete')

"""
import transformers
tokenizer = transformers.BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")
print(tokenizer.tokenize("お腹が空いた"))
"""